# ***Alignements structuraux de protéines***

Les alignements de séquences de protéines et d’acides nucléiques sont un outil essentiel
en bioinformatique, permettant de mettre en évidence des positions conservées et de faire des
hypothèses structurales, évolutionnistes ou fonctionnelles. Lorsque la structure des biomolécules
est connue, cette information peut être prise en compte pour améliorer la qualité des alignements
de séquences. On se propose d’écrire un programme d’alignements de séquences de protéines
utilisant l’information structurale et d’évaluer sa performance.

Les séquences que nous allons utiliser sont en format FASTA

## Alignement de sequences
On va écrire un programme pour aligner deux séquences, en utilisant l'algorithme Needleman-Wunch

Une première étape : on va faire ca pour un cout fixe d'identité et de substitution

In [ ]:
gapPenalty=-4
identity=1
substitution=-1

In [ ]:
def cost_function_example(x,y):
  """
  calcule le cout de l'alignement des car x et y
  entrée : deux car x et y
  sortie : un int représentant le cout de l'alignement de x et y
  """

  if x==y:
    return identity
  elif x=="-" or y=="-":
    return gapPenalty
  else:
    return substitution

On va faire une fonction qui calcule la matrice des couts, et qui garde en mémoire au fur et à mesure le chemin pris et l'alignement construit

In [ ]:
def costMatrix(str1, str2, cost_fun):

  ''' Construction de la matrice de couts
    et de la matrice des alignements construits petit à petit
    et de la matrice des flèches

  Entrée : les deux chaines de caractères correspondant aux séquences et la fonction de cout

  Sortie : la matrice des couts scoreM et la matrice des sequences alignées seqM et la matrice des flèches arrowM
      scoreM[i][j] = (cout : int) de taille (len(str2)+1) x (len(str1)+1)
        avec scoreM[i][j]= max(scoreM[i-1][j-1]+(cout(str1[i], str2[j])), scoreM[i-1][j]+cout(str1[i],"-"), scoreM[i][j-1]+cout("-", str2[j]))

      seqM[i][j] = [("","")] liste de tuples de strings  de taille (len(str2)+1) x (len(str1)+1)
      chaque tuple de string contient un alignement possible (et le moins couteux possible) des deux séquences.
      Il peut y avoir plusieurs tuples s'il y a plusieurs chemins

      arrowM[i][j] = ["↖"] liste de tous les chemins possible, sous symbole de flèche
        de taille (len(str2)+1) x (len(str1)+1)

  '''
  scoreM = [[0 for i in range(len(str1)+1)] for j in range(len(str2)+1)]
  seqM= [[[] for i in range(len(str1)+1)] for j in range(len(str2)+1)]
  arrowM= [[[] for i in range(len(str1)+1)] for j in range(len(str2)+1)]
  for i in range (len(str2)+1):
    for j in range (len(str1)+1):
      if i==0 and j==0: #cas de base
        scoreM[i][j]=0
        seqM[i][j].append(("",""))
        arrowM[i][j].append(" ")
      elif i==0:
        scoreM[i][j] = scoreM[i][j-1]+cost_fun("-", str1[j-1])
        arrowM[i][j].append("←")
        for k in range (len(seqM[i][j-1])):
          seqM[i][j].append((seqM[i][j-1][k][0]+str1[j-1], seqM[i][j-1][k][1]+"_"))
      elif j==0:
        scoreM[i][j] = scoreM[i-1][j]+cost_fun(str2[i-1], "-")
        arrowM[i][j].append("↑")
        for k in range(len(seqM[i-1][j])):
          seqM[i][j].append((seqM[i-1][j][k][0]+"_", seqM[i-1][j][k][1]+str2[i-1]))
      else :
        cout = max (scoreM[i-1][j-1]+(cost_fun(str2[i-1],str1[j-1])), scoreM[i-1][j]+cost_fun(str2[i-1],"-"), scoreM[i][j-1]+cost_fun("-",str1[j-1]))
        scoreM[i][j] = cout
        if cout == scoreM[i-1][j-1]+cost_fun(str2[i-1],str1[j-1]):
          arrowM[i][j].append("↖")
          for k in range(len(seqM[i-1][j-1])):
            seqM[i][j].append((seqM[i-1][j-1][k][0]+str1[j-1], seqM[i-1][j-1][k][1]+str2[i-1]))
        if cout == scoreM[i-1][j]+cost_fun(str2[i-1],"-"): #On met pas un elif car plusieurs cas sont possibles
          arrowM[i][j].append("↑")
          for k in range(len(seqM[i-1][j])):
            seqM[i][j].append((seqM[i-1][j][k][0]+"_", seqM[i-1][j][k][1]+str2[i-1]))
        if cout == scoreM[i][j-1]+cost_fun("-",str1[j-1]):
          arrowM[i][j].append("←")
          for k in range(len(seqM[i][j-1])):
            seqM[i][j].append((seqM[i][j-1][k][0]+str1[j-1], seqM[i][j-1][k][1]+"_"))

  return scoreM, seqM, arrowM

In [ ]:
##Test du code prédédent

print(costMatrix("CHAT","CAT",cost_function_example))

([[0, -4, -8, -12, -16], [-4, 1, -3, -7, -11], [-8, -3, 0, -2, -6], [-12, -7, -4, -1, -1]], [[[('', '')], [('C', '_')], [('CH', '__')], [('CHA', '___')], [('CHAT', '____')]], [[('_', 'C')], [('C', 'C')], [('CH', 'C_')], [('CHA', 'C__')], [('CHAT', 'C___')]], [[('__', 'CA')], [('C_', 'CA')], [('CH', 'CA')], [('CHA', 'C_A')], [('CHAT', 'C_A_')]], [[('___', 'CAT')], [('C__', 'CAT')], [('C_H', 'CAT'), ('CH_', 'CAT')], [('CHA', 'CAT')], [('CHAT', 'C_AT')]]], [[[' '], ['←'], ['←'], ['←'], ['←']], [['↑'], ['↖'], ['←'], ['←'], ['←']], [['↑'], ['↑'], ['↖'], ['↖'], ['←']], [['↑'], ['↑'], ['↖', '↑'], ['↖'], ['↖']]])


In [ ]:
def seq_align(str1, str2, cost_fun):
  '''
  Renvoie les différents alignements possibles ainsi que le cout
  Entrée : les deux séquences à comparer (strings)
  Sortie : le cout du meilleur alignement (int)
        et les différents alignements possibles (liste de stuple de strings) où un tuple est un alignement des deux strings
  '''
  scoreM,seqM,arrowM=costMatrix(str1,str2,cost_fun)
  #print(scoreM,seqM)
  return scoreM[len(str2)][len(str1)], (seqM[len(str2)][len(str1)])


In [ ]:
#seq_align("CHAT","CAT")
seq_align("AAAG","ACG", cost_function_example)

(-3, [('AAAG', '_ACG'), ('AAAG', 'A_CG'), ('AAAG', 'AC_G')])

In [ ]:
def representation(str1, str2, costM,arrowM):
  repr = ""

  header = " " + "      ".join(f"{c}  " for c in " -" + str1) + "\n"

  repr += header

  for i, line in enumerate(costM):

    row = f"{'-' if i == 0 else str2[i-1] if i > 0 else ''}    "

    for j, cell in enumerate(line):
      row+="  "
      row += "".join(
        arrowM[i][j][k]
        for k in range(len(arrowM[i][j]))
      )
      row+=f"  {cell or costM[i][j]}  "
    row += "\n"

    repr += row

  print(repr)

In [ ]:
costM, seqqM, arrowM=costMatrix("AAAG","ACG",cost_function_example)
representation("AAAG","ACG",costM,arrowM)

          -        A        A        A        G  
-         0    ←  -4    ←  -8    ←  -12    ←  -16  
A      ↑  -4    ↖  1    ↖←  -3    ↖←  -7    ←  -11  
C      ↑  -8    ↑  -3    ↖  0    ↖←  -4    ↖←  -8  
G      ↑  -12    ↑  -7    ↖↑  -4    ↖  -1    ↖  -3  



#Scéance 2 : Alignement de plusieurs séquences

On va adapter notre programme afin de permettre d'aligner un nombre arbritraire de séquences. Cela se produit en plusieurs étapes:



*   *Etape 1* : Calcul de distances entre les séquences deux à deux
*   *Etape 2* : Construction d'un arbre hierarchique, à partir de ces distances
*   *Etape 3* : Alignements entre blocs déjà alignés




*Etape 1*

In [ ]:
def mat_distances(liste_de_sequences, cost_fun):
  """renvoie une matrice de distances deux à deux des séquences de la liste
  Entrée : une liste de string qui sont les séquences à comparer
  Sortie : une matrice de distances deux à deux des séquences de la liste"""
  matD=[[0 for j in range(len(liste_de_sequences))] for i in range(len(liste_de_sequences))]
  for i in range(len(liste_de_sequences)):
    for j in range(len(liste_de_sequences)):
      if j>=i:
        matD[i][j]=seq_align(liste_de_sequences[i], liste_de_sequences[j], cost_fun)[0]
        matD[j][i]=matD[i][j]
  return matD

In [ ]:
#test

print(mat_distances(["CHAT","CAT","HER"],cost_function_example))

[[4, -1, -5], [-1, 3, -3], [-5, -3, 3]]


*Etape 2 : construction de l'arbre de clusters*

On aura besoin de définir les structures d'arbres

In [ ]:
class Tree:
  def __init__(self, val = None, left=None, right=None):
    if val != None:
        self.val = val
    else:
        self.val = None

    if left != None:
        self.left = left
    else:
        self.left = None

    if right != None:
        self.right = right
    else:
        self.right = None

Avec une fonction de représentation

In [ ]:
def print_tree(node, level=0, prefix="Root: "):
    """
    Représente un arbre binaire de manière hiérarchique.
    :param node: Le nœud actuel (de type Tree)
    :param level: Niveau d'indentation actuel
    :param prefix: Préfixe à afficher avant chaque nœud
    """
    if node is not None:
        print(" " * (level * 4) + prefix + str(node.val))
        print_tree(node.left, level + 1, "Left: ")
        print_tree(node.right, level + 1, "Right: ")
    else:
        print(" " * (level * 4) + prefix + "None")

In [ ]:
def inter_cluster_dist(list_indices1,list_indices2,distanceMatrix):
  """renvoie la distance entre le cluster 1 et 2
  Entrée : deux clusters, étant des arbres, chaque noeud étant un tuple (indice, séquence associée)
          une matrice de distance qui associe aux indices concernées la distance entre les séquences correspondantes
  Sortie : float, la distance entre les deux clusters"""
  dist=0
  for i in list_indices1:
    for j in list_indices2:
      dist+=distanceMatrix[i][j]
  return dist/(len(list_indices1)*len(list_indices2))

In [ ]:
def UPGMA(liste_de_sequences, cost_fun):
  """renvoie l'arbre de clusters associé à la liste de séquence
  entrée : une liste de séquence et la fonction de cout de l'alignement
  sortie : un arbre de clusters"""

  matD=mat_distances(liste_de_sequences, cost_fun) #matrice de distance entre chaque paire de séquence
  clusters=[Tree(val=[i]) for i in range(len(liste_de_sequences))] #liste de cluster, chaque cluster étant un arbre
  mind = float('inf')
  while len(clusters)>1:
    min=matD[0][1]
    min_i=0
    min_j=1
    for i in range(len(clusters)):
      for j in range(i+1,len(clusters)):
        dij=inter_cluster_dist(clusters[i].val,clusters[j].val,matD)
        if dij<mind :
          mind=dij
          min_i=i
          min_j=j
    indices_new_cluster=clusters[min_i].val+clusters[min_j].val
    new_cluster=Tree(indices_new_cluster, left=clusters[min_i], right=clusters[min_j])
    clusters.pop(min_j)
    clusters.pop(min_i)
    clusters.append(new_cluster)
  return clusters

In [ ]:
tree_example =UPGMA(["CHAT","CAT","HER"],cost_function_example)[0]
print_tree(tree_example)

Root: [1, 0, 2]
    Left: [1]
        Left: None
        Right: None
    Right: [0, 2]
        Left: [0]
            Left: None
            Right: None
        Right: [2]
            Left: None
            Right: None


*Etape 3 : alignement*

In [ ]:
def cost_function_block(list1, list2, cost_fun):
  """ renvoie le cout de l'alignement entre deux listes de caractères
  Entrée : deux listes de caractères et la fonction de cout par base
  Sortie : le cout associé"""
  cost=0
  n=len(list1)
  m=len(list2)
  for i in range(n):
    for j in range(m):
      cost+=cost_fun(list1[i],list2[j])
  return cost/(n*m)

In [ ]:
#test
cost_function_block(["T","T","T"],["-","-"],cost_function_example)

-4.0

On adapte notre fonction needleman wunch

In [ ]:
import numpy

In [ ]:
def costMatrixMultiple(block1, block2, cost_fun):
    """
    Compute the Needleman-Wunsch dynamic programming matrix concerning blocks of strings already aligned

    Entrée : les deux chaines de caractères correspondant aux séquences et la fonction de cout

    Returns:
      The dynamic programming matrix
    """

    n=len(block1[0])
    m=len(block2[0])

    # construct DP matrix (use a numpy matrix!)
    D = numpy.zeros((m+1,n+1), dtype='int')

    # fill matrix
    for i in range(0,m+1):
      for j in range(0,n+1):
        if i==0 and j==0:
          D[i,j]=0
        elif i==0:
          D[i,j]=D[i,j-1]+cost_function_block([('-') for k in range(len(block1))],[(block2[k][j-1]) for k in range (len(block2))], cost_fun)
        elif j==0:
          D[i,j]=D[i-1,j]+cost_function_block([(block1[k][j-1])for k in range(len(block1))],[('-') for k in range (len(block2))], cost_fun)
        else :
          D[i,j]=max(D[i-1,j]+cost_function_block([(block1[k][j-1])for k in range(len(block1))],[('-') for k in range (len(block2))], cost_fun),
                     D[i,j-1]+cost_function_block([('-') for k in range(len(block1))],[(block2[k][j-1]) for k in range (len(block2))], cost_fun),
                     D[i-1,j-1]+cost_function_block([(block1[k][j-1]) for k in range(len(block1))],[(block2[k][j-1]) for k in range (len(block2))], cost_fun))
    return(D)


In [ ]:
Mat= (costMatrixMultiple(["CHAT","C-AT","CH-T"], ["-HAT"], cost_function_example))
print(Mat)

[[  0   1  -3  -7 -11]
 [ -4  -3   0  -3  -6]
 [ -8  -7  -2   0  -2]
 [-12 -11  -4  -2   1]
 [-16 -15  -6  -4  -1]]


In [ ]:
def trace_back_needleman_wunsch_multiple(block1, block2, cost_fun, D):
    """
    Determine the alignment of two already aligned blocks given a dynamic programming matrix mat
    Args:
      block1: list of str, first list of aligned sequences
      block2: list of str, second list of aligned sequences
      cost_fun: cost function
      mat: The dynamic programming table
    Returns:
      Two block of strings with "-" symbols: The alignment between block1 and block2
    """
    # perform trace back, construct alignment blocks of string aa and ab
    aa = [("") for i in range (len(block1))]
    bb = [("") for i in range (len(block2))]
    i=len(block1[0])
    j=len(block2[0])

    while i>0 or j>0:

      if i==0:
        for k in range (len(block1)):
          aa[k]+='-'
        for k in range(len(block2)):
          bb[k]+=(block2[k][j-1])
        j-=1

      elif j==0:
        for k in range (len(block1)):
          aa[k]+=(block1[k][i-1])
        for k in range(len(block2)):
          bb[k]+='-'
        i-=1


      else :
        if D[i,j]==D[i-1,j-1]+cost_function_block([(block1[k][i-1]) for k in range(len(block1))],[(block2[k][j-1]) for k in range (len(block2))], cost_fun):
          for k in range (len(block1)):
            aa[k]+=(block1[k][i-1])
          for k in range(len(block2)):
            bb[k]+=(block2[k][j-1])
          i-=1
          j-=1

        elif D[i,j]==D[i-1,j]+cost_function_block([(block1[k][i-1])for k in range(len(block1))],[('-') for k in range (len(block2))], cost_fun):
          print("cas2")
          print([(block1[k][j-1])for k in range(len(block1))],[('-') for k in range (len(block2))])
          for k in range (len(block1)):
            aa[k]+=(block1[k][i-1])
          for k in range(len(block2)):
            bb[k]+='-'
          i-=1

        else :
          for k in range (len(block1)):
            aa[k]+='-'
          for k in range(len(block2)):
            bb[k]+=(block2[k][j-1])
          j-=1

    for k in range (len(block1)):
      aa[k]=aa[k][::-1]
    for k in range(len(block2)):
      bb[k]=bb[k][::-1]
    print(aa, bb)

    return D[len(block1[0]), len(block2[0])]

In [ ]:
trace_back_needleman_wunsch_multiple(["CHAT","C-AT","CH-T"], ["-HAT"], cost_function_example, Mat)

['CHA---T', 'C-A---T', 'CH----T'] ['----HAT']


-1